In [ ]:
!pip install torch torchvision  # Required for YOLOv5
!pip install opencv-python  # Required for image processing


In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!git clone https://github.com/ultralytics/yolov5  # Clone the YOLOv5 repository
%cd yolov5
!pip install -r requirements.txt  # Install YOLOv5 dependencies


In [ ]:
import torch
from torchvision.models import load_state_dict_from_url

model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5s.pt', force_reload=True)  # YOLOv5 model
model.eval()  # Set the model to evaluation mode


In [ ]:
import os
import cv2
import torch

# Define the path to the CNN folder in your Google Drive
cnn_folder_path = '/content/drive/MyDrive/CNN'

# Define the directory paths for the normal and dark folders
normal_folder_path = os.path.join(cnn_folder_path, 'normal')
dark_folder_path = os.path.join(cnn_folder_path, 'dark')

# Define the image directories
image_dirs = {
    'normal_train': os.path.join(normal_folder_path, 'train'),
    'dark_train': os.path.join(dark_folder_path, 'train')
}

# Detect humans with YOLOv5 and crop images
for image_dir_key, image_dir_path in image_dirs.items():
    print('Processing images in:', image_dir_key)
    
    for class_name in ['sit', 'walk', 'fall']:
        class_dir = os.path.join(image_dir_path, class_name)

        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)

            # Load the image
            image = cv2.imread(image_path)

            # Detect objects using YOLOv5
            results = model(image)

            # Extract bounding boxes and crop images
            for detection in results.pandas().xyxy[0].itertuples():
                xmin, ymin, xmax, ymax, _, _, _, _ = detection[1:]
                cropped_image = image[int(ymin):int(ymax), int(xmin):int(xmax)]

                # Save the cropped image or perform further processing
                cropped_image_path = os.path.join(image_dir_path, 'cropped', class_name, image_name)
                os.makedirs(os.path.dirname(cropped_image_path), exist_ok=True)
                cv2.imwrite(cropped_image_path, cropped_image)


In [ ]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

def load_dataset(dataset_path):
    X = []
    y = []

    for class_name in os.listdir(dataset_path):
        class_dir = os.path.join(dataset_path, class_name)
        
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)
            
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale if needed
            
            X.append(image)
            y.append(class_name)
    
    X = np.array(X)
    y = np.array(y)
    
    return X, y

# Define the path to your cropped images dataset
dataset_path = '/content/drive/MyDrive/CNN/normal_train/cropped'

# Load the dataset
X, y = load_dataset(dataset_path)

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from skimage.feature import hog

def extract_hog_features(images):
    features = []

    for image in images:
        feature_vector = hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False)
        features.append(feature_vector)
    
    return np.array(features)

# Extract HOG features from the training and validation sets
X_train_hog = extract_hog_features(X_train)
X_val_hog = extract_hog_features(X_val)


In [ ]:
from sklearn.svm import SVC

# Create and train the SVM classifier
hog_classifier = SVC()
hog_classifier.fit(X_train_hog, y_train)


In [ ]:
from sklearn.metrics import accuracy_score

# Predict on the validation set
y_pred = hog_classifier.predict(X_val_hog)

# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred)
print('Accuracy:', accuracy)


In [ ]:
# Define the path to save the HOG classifier model
hog_model_path = '/content/drive/MyDrive/CNN/hog_classifier.pkl'

# Save the HOG classifier model
import joblib
joblib.dump(hog_classifier, hog_model_path)


In [ ]:
import cv2
import joblib

# Load the trained HOG classifier model
hog_model_path = '/content/drive/MyDrive/CNN/hog_classifier.pkl'
hog_classifier = joblib.load(hog_model_path)


In [ ]:
# Detect humans with YOLOv5 and feed cropped images to HOG classifier
for detection in results.pandas().xyxy[0].itertuples():
    xmin, ymin, xmax, ymax, _, _, _, _ = detection[1:]
    cropped_image = image[int(ymin):int(ymax), int(xmin):int(xmax)]

    # Resize the cropped image to match the input size of the HOG classifier
    resized_image = cv2.resize(cropped_image, (64, 128))  # Adjust the size as needed

    # Convert the resized image to grayscale if needed
    resized_image_gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

    # Extract HOG features from the resized image
    hog_features = hog(resized_image_gray, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False)

    # Reshape the feature vector to match the expected input shape of the HOG classifier
    hog_features = hog_features.reshape(1, -1)

    # Make prediction using the HOG classifier
    action_prediction = hog_classifier.predict(hog_features)

    # Print the predicted action
    print("Action Prediction:", action_prediction)


In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

def detect_and_classify(image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Detect objects using YOLOv5
    results = model(image)

    # Process each detection
    for detection in results.pandas().xyxy[0].itertuples():
        xmin, ymin, xmax, ymax, _, _, _, _ = detection[1:]
        cropped_image = image[int(ymin):int(ymax), int(xmin):int(xmax)]

        # Resize the cropped image to match the input size of the HOG classifier
        resized_image = cv2.resize(cropped_image, (64, 128))  # Adjust the size as needed

        # Convert the resized image to grayscale if needed
        resized_image_gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

        # Extract HOG features from the resized image
        hog_features = hog(resized_image_gray, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False)

        # Reshape the feature vector to match the expected input shape of the HOG classifier
        hog_features = hog_features.reshape(1, -1)

        # Make prediction using the HOG classifier
        action_prediction = hog_classifier.predict(hog_features)

        # Print the predicted action
        print("Action Prediction:", action_prediction)

        # Display the image with bounding box and action prediction
        cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 2)
        cv2.putText(image, action_prediction[0], (int(xmin), int(ymin) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Convert BGR to RGB for plotting
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Plot the image
    plt.figure(figsize=(10, 10))
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.show()

# Define the paths to the test directories
normal_test_dir = '/content/drive/MyDrive/CNN/normal/test'
dark_test_dir = '/content/drive/MyDrive/CNN/dark/test'

# Test images in the normal test directory
print('Testing images in normal test directory:')
for image_name in os.listdir(normal_test_dir):
    image_path = os.path.join(normal_test_dir, image_name)
    print('Image:', image_name)
    detect_and_classify(image_path)

# Test images in the dark test directory
print('Testing images in dark test directory:')
for image_name in os.listdir(dark_test_dir):
    image_path = os.path.join(dark_test_dir, image_name)
    print('Image:', image_name)
    detect_and_classify(image_path)


In [ ]:
from sklearn.metrics import precision_recall_curve, confusion_matrix, f1_score, roc_curve, roc_auc_score
import seaborn as sns

# Define a function to calculate precision-recall curve and plot it
def plot_precision_recall(y_true, y_pred_prob):
    precision, recall, _ = precision_recall_curve(y_true, y_pred_prob)
    plt.figure(figsize=(8, 6))
    plt.plot(recall, precision)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.grid(True)
    plt.show()

# Define a function to calculate and plot the confusion matrix
def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

# Define a function to calculate F1-confidence curve and plot it
def plot_f1_confidence_curve(y_true, y_pred_prob):
    thresholds = np.arange(0, 1.05, 0.05)
    f1_scores = []
    confidences = []

    for threshold in thresholds:
        y_pred = (y_pred_prob >= threshold).astype(int)
        f1_scores.append(f1_score(y_true, y_pred))
        confidences.append(np.mean(y_pred_prob))

    plt.figure(figsize=(8, 6))
    plt.plot(confidences, f1_scores)
    plt.xlabel('Confidence')
    plt.ylabel('F1 Score')
    plt.title('F1-Confidence Curve')
    plt.grid(True)
    plt.show()

# Define a function to calculate ROC curve and plot it
def plot_roc_curve(y_true, y_pred_prob):
    fpr, tpr, _ = roc_curve(y_true, y_pred_prob)
    auc = roc_auc_score(y_true, y_pred_prob)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve (AUC = {:.3f})'.format(auc))
    plt.grid(True)
    plt.show()

# Define the paths to the test directories
normal_test_dir = '/content/drive/MyDrive/CNN/normal/test'
dark_test_dir = '/content/drive/MyDrive/CNN/dark/test'

# Initialize lists to store true labels and predicted probabilities
y_true = []
y_pred_prob = []

# Test images in the normal test directory
print('Testing images in normal test directory:')
for image_name in os.listdir(normal_test_dir):
    image_path = os.path.join(normal_test_dir, image_name)
    print('Image:', image_name)
    detect_and_classify(image_path)  # Modify this line to append true labels and predicted probabilities

# Test images in the dark test directory
print('Testing images in dark test directory:')
for image_name in os.listdir(dark_test_dir):
    image_path = os.path.join(dark_test_dir, image_name)
    print('Image:', image_name)
    detect_and_classify(image_path)  # Modify this line to append true labels and predicted probabilities

# Convert the lists to numpy arrays
y_true = np.array(y_true)
y_pred_prob = np.array(y_pred_prob)

# Calculate and plot precision-recall curve
plot_precision_recall(y_true, y_pred_prob)

# Calculate and plot confusion matrix
y_pred = (y_pred_prob >= 0.5).astype(int)  # Adjust the threshold as needed
plot_confusion_matrix(y_true, y_pred)

# Calculate and plot F1-confidence curve
plot_f1_confidence_curve(y_true, y_pred_prob)

# Calculate and plot ROC curve
plot_roc_curve(y_true, y_pred_prob)